# Environment

### Imports

In [7]:
import gym
from gym import Env, spaces
import numpy as np
from gymnasium.wrappers import FlattenObservation

# Utilidad
import json
import copy # Para copiar las variables originales a las que el ambiente edite para cuando se haga un reset
import random

### Data Example

#### Estudiantes

In [53]:
estudiantes_json = [
    {
        "nombre": "Javier Ahumada",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Santiago Lopez",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            0,
            1,
            2
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            3,
            1,
            2
        ]
    }
]

#### Desafíos

In [54]:
desafios_json = [
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    }
]

# desafios = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]

### Funciones útiles

### Definición

In [3]:
class Estudiante(object):
    def __init__(self, id, preferencias = []):
        self.id = id
        self.preferencias = preferencias # IDs de desafíos

class Desafio(object):
    def __init__(self, id, estudiantes : list = []):
        self.id = id
        self.estudiantes = estudiantes # IDs de estudiantes

    def push(self, estudiante):
        self.estudiantes.append(estudiante)

In [11]:
num_estudiantes = 3
num_desafios = 4
matriz_preferencias = [[ num_desafios for _ in range(num_desafios)] for _ in range(num_estudiantes)]
matriz_escogido = [[ 1 for _ in range(num_desafios)] for _ in range(num_estudiantes)]

for fila in matriz_escogido:
    print(fila)

[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]


In [9]:
preferencias = [
    [1,2,3,0],
    [0,3,2,1],
    [3,2,1,0]
]

escogidos = [
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0]
]

In [51]:
[[ 2 for _ in range(num_desafios)] for _ in range(num_estudiantes)]

[[2, 2, 2, 2], [2, 2, 2, 2], [2, 2, 2, 2]]

In [37]:
print(list(range(len(preferencias))))

[0, 1, 2]


In [66]:
np.array([[[ 2 for _ in range(num_desafios)] for _ in range(num_estudiantes)],[[ num_desafios for _ in range(num_desafios)] for _ in range(num_estudiantes)]])

array([[[2, 2, 2, 2],
        [2, 2, 2, 2],
        [2, 2, 2, 2]],

       [[4, 4, 4, 4],
        [4, 4, 4, 4],
        [4, 4, 4, 4]]])

In [8]:
[0] * (10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
class OrganizationEnvironment(gym.Env):
    def __init__(self, preferencias = None):
        super(OrganizationEnvironment, self).__init__()

        if(preferencias == None):
            self.preferencias = self.get_preferencias()
        else:
            self.preferencias = preferencias

        self.num_estudiantes = len(self.preferencias)
        self.num_desafios = len(self.preferencias[0])

        self.num_preferencias = 0
        for i in self.preferencias:
            for j in i:
                if j > self.num_preferencias:
                    self.num_preferencias = j


        # Espacio de observación
        self.state = [[ 0 for _ in range(self.num_desafios)] for _ in range(self.num_estudiantes)]
        self.current_ids = list(range(len(self.preferencias)))
        self.reset()

        # self.observation_space_preferencias = [[ self.num_desafios for _ in range(self.num_desafios)] for _ in range(self.num_estudiantes)]
        # self.observation_space_escogido = [[ 2 for _ in range(self.num_desafios)] for _ in range(self.num_estudiantes)]
        # self.observation_space = spaces.MultiDiscrete(np.array([self.observation_space_escogido,self.observation_space_preferencias]),  seed=42)
        self.observation_space = spaces.MultiDiscrete([2] * (self.num_estudiantes * self.num_desafios) + \
                                                      [self.num_desafios] * (self.num_estudiantes * self.num_desafios))

        # Definición de espacio de acción
        self.action_space = spaces.MultiDiscrete([self.num_estudiantes, self.num_desafios])


    def get_preferencias(self):
        num_estudiantes = 10
        num_desafios = 4

        preferencias_totales = []

        for i in range(num_estudiantes):
            preferencias_estudiante = [0] * (num_desafios) # [0,0,0,0...]

            numero_preferencia = 1

            while True:
                desafio = random.randint(0, num_desafios - 1)
                if(preferencias_estudiante[desafio] == 0):
                    preferencias_estudiante[desafio] = numero_preferencia
                    numero_preferencia += 1
                if(numero_preferencia > 3):
                    break
            preferencias_totales.append(preferencias_estudiante)

        print("\t  Preferencias")
        for index, fila in enumerate(preferencias_totales):
            print(f"Alumno {index}: ",fila)


        return preferencias_totales

    def render(self):
        print("\t  Desafíos")
        for index, fila in enumerate(self.state):
            print(f"Alumno {index}: ",fila)

    def reset(self):

        last_observation = self._get_observation()

        self.preferencias = self.get_preferencias()

        self.num_estudiantes = len(self.preferencias)
        self.num_desafios = len(self.preferencias[0])

        self.num_preferencias = 0
        for i in self.preferencias:
            for j in i:
                if j > self.num_preferencias:
                    self.num_preferencias = j


        # Espacio de observación
        self.state = [[ 0 for _ in range(self.num_desafios)] for _ in range(self.num_estudiantes)]
        self.current_ids = list(range(len(self.preferencias)))

        return last_observation

    def step(self, action):
        assert self.action_space.contains(action), "Invalid Action"

        id_estudiante = action[0] # 3
        id_desafio = action[1]

        real_id = 0

        for ids in range(len(self.current_ids)):
            if(self.current_ids[ids] == id_estudiante):
                self.current_ids[ids] = -1
                real_id = ids

        for ids in range(len(self.current_ids)):
            if(self.current_ids[ids] > id_estudiante):
                self.current_ids[ids] = self.current_ids[ids] - 1

        self.state[real_id][id_desafio] = 1
        done = self.check_done()

        if(not done):
            contador_escogidos = 0
            for estudiante in self.state:
                for desafio in estudiante:
                    contador_escogidos += desafio

            self.action_space = spaces.MultiDiscrete([self.num_estudiantes - contador_escogidos, self.num_desafios])

        reward = self.get_reward([real_id, id_desafio])


        # observation = [self.state, self.preferencias]
        observation = self._get_observation()
        info = {}

        return observation, reward, done, info

    def _get_observation(self):
        # Aplanar el estado y las preferencias para ajustarse al espacio de observación
        return [self.state[i][j] for i in range(self.num_estudiantes) for j in range(self.num_desafios)] + \
               [self.preferencias[i][j] for i in range(self.num_estudiantes) for j in range(self.num_desafios)]

    def get_reward(self, action):
        id_estudiante = action[0]
        id_desafio = action[1]

        reward =    self.num_preferencias - self.preferencias[id_estudiante][id_desafio] + 1 \
                    if self.preferencias[id_estudiante][id_desafio] != 0 \
                    else 0

        # Implementar función de reward
        return reward

    def check_done(self):
        # Condición para finalizar el episodio
        contador_de_estudiantes = 0
        for estudiante in self.state:
            for seleccion in estudiante:
                contador_de_estudiantes += seleccion

        if(contador_de_estudiantes == self.num_estudiantes):
            return True

        return False

env = OrganizationEnvironment()

	  Preferencias
Alumno 0:  [1, 3, 2, 0]
Alumno 1:  [1, 3, 0, 2]
Alumno 2:  [0, 3, 1, 2]
Alumno 3:  [3, 1, 0, 2]
Alumno 4:  [2, 0, 3, 1]
Alumno 5:  [0, 2, 3, 1]
Alumno 6:  [3, 1, 0, 2]
Alumno 7:  [1, 2, 3, 0]
Alumno 8:  [1, 2, 0, 3]
Alumno 9:  [2, 3, 0, 1]
	  Preferencias
Alumno 0:  [1, 2, 3, 0]
Alumno 1:  [1, 3, 2, 0]
Alumno 2:  [0, 3, 1, 2]
Alumno 3:  [1, 3, 0, 2]
Alumno 4:  [1, 2, 3, 0]
Alumno 5:  [2, 0, 1, 3]
Alumno 6:  [1, 0, 2, 3]
Alumno 7:  [1, 2, 0, 3]
Alumno 8:  [0, 2, 3, 1]
Alumno 9:  [3, 0, 1, 2]


In [31]:
env.reset()
while True:
    action = env.action_space.sample()

    obs, reward, done, info = env.step(action)

    env.render()

    if done == True:
        break

# print(binary_to_dict2(obs))

	  Preferencias
Alumno 0:  [2, 1, 0, 3]
Alumno 1:  [3, 1, 2, 0]
Alumno 2:  [3, 1, 2, 0]
Alumno 3:  [0, 2, 3, 1]
Alumno 4:  [3, 1, 0, 2]
Alumno 5:  [2, 1, 0, 3]
Alumno 6:  [3, 1, 0, 2]
Alumno 7:  [1, 3, 0, 2]
Alumno 8:  [0, 2, 1, 3]
Alumno 9:  [3, 1, 2, 0]
	  Desafíos
Alumno 0:  [0, 0, 0, 0]
Alumno 1:  [0, 0, 0, 0]
Alumno 2:  [0, 0, 0, 0]
Alumno 3:  [0, 0, 0, 0]
Alumno 4:  [0, 0, 0, 0]
Alumno 5:  [1, 0, 0, 0]
Alumno 6:  [0, 0, 0, 0]
Alumno 7:  [0, 0, 0, 0]
Alumno 8:  [0, 0, 0, 0]
Alumno 9:  [0, 0, 0, 0]
	  Desafíos
Alumno 0:  [0, 0, 0, 0]
Alumno 1:  [0, 0, 0, 0]
Alumno 2:  [0, 0, 0, 0]
Alumno 3:  [0, 0, 0, 0]
Alumno 4:  [0, 0, 0, 0]
Alumno 5:  [1, 0, 0, 0]
Alumno 6:  [0, 0, 0, 0]
Alumno 7:  [0, 1, 0, 0]
Alumno 8:  [0, 0, 0, 0]
Alumno 9:  [0, 0, 0, 0]
	  Desafíos
Alumno 0:  [0, 0, 0, 0]
Alumno 1:  [0, 0, 0, 0]
Alumno 2:  [0, 0, 0, 0]
Alumno 3:  [0, 0, 0, 0]
Alumno 4:  [0, 0, 0, 0]
Alumno 5:  [1, 0, 0, 0]
Alumno 6:  [0, 0, 0, 0]
Alumno 7:  [0, 1, 0, 0]
Alumno 8:  [0, 0, 0, 0]
Alumno 9:  [

In [19]:
env.observation_space.sample()

array([ 1,  1,  1, ...,  8, 28, 13])

In [33]:
from stable_baselines3 import PPO

model = PPO("MlpPolicy", env).learn(total_timesteps=100_000)

obs = env.reset()

while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()
    print(reward)
    if done == True:
        break

# print(binary_to_dict(obs))

/home/javo/.local/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


	  Preferencias
Alumno 0:  [2, 1, 0, 3]
Alumno 1:  [3, 1, 2, 0]
Alumno 2:  [1, 0, 3, 2]
Alumno 3:  [1, 0, 2, 3]
Alumno 4:  [2, 0, 3, 1]
Alumno 5:  [0, 3, 1, 2]
Alumno 6:  [3, 2, 1, 0]
Alumno 7:  [0, 1, 2, 3]
Alumno 8:  [0, 3, 1, 2]
Alumno 9:  [1, 3, 2, 0]
	  Preferencias
Alumno 0:  [3, 1, 2, 0]
Alumno 1:  [2, 3, 0, 1]
Alumno 2:  [3, 0, 2, 1]
Alumno 3:  [3, 1, 0, 2]
Alumno 4:  [1, 2, 3, 0]
Alumno 5:  [3, 0, 1, 2]
Alumno 6:  [3, 0, 1, 2]
Alumno 7:  [2, 0, 3, 1]
Alumno 8:  [0, 1, 2, 3]
Alumno 9:  [3, 0, 2, 1]
	  Preferencias
Alumno 0:  [2, 3, 1, 0]
Alumno 1:  [2, 1, 3, 0]
Alumno 2:  [3, 0, 2, 1]
Alumno 3:  [2, 0, 1, 3]
Alumno 4:  [1, 0, 3, 2]
Alumno 5:  [2, 1, 3, 0]
Alumno 6:  [0, 1, 2, 3]
Alumno 7:  [3, 1, 0, 2]
Alumno 8:  [2, 3, 0, 1]
Alumno 9:  [3, 0, 1, 2]
	  Preferencias
Alumno 0:  [2, 0, 3, 1]
Alumno 1:  [3, 1, 0, 2]
Alumno 2:  [3, 0, 1, 2]
Alumno 3:  [1, 3, 2, 0]
Alumno 4:  [2, 0, 3, 1]
Alumno 5:  [1, 3, 2, 0]
Alumno 6:  [0, 3, 2, 1]
Alumno 7:  [3, 2, 1, 0]
Alumno 8:  [1, 2, 0, 3]
